# Extract ground observations for Snow Melt Timing Hackweek Project
## Set Up Computing Environment

In [7]:
# standard imports
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt 
import datetime

# some mapping widgets
import ipyleaflet
from ipyleaflet import Map, GeoData, Rectangle, basemaps, LayersControl, basemap_to_tiles, TileLayer, SplitMapControl, Polygon, MagnifyingGlass
import ipywidgets

# database imports
from snowexsql.db import get_db
from snowexsql.data import PointData, LayerData, ImageData, SiteData
from snowexsql.conversions import query_to_geopandas, query_to_pandas

# Query database for 2020 Grand Mesa Time Series Sites

In [8]:
# load the database
db_name = 'snow:hackweek@db.snowexdata.org/snowex'
engine, session = get_db(db_name)

print('SnowEx Database successfully loaded!')

SnowEx Database successfully loaded!


In [9]:
# list of sites, these are from the 2020 Time Series Campaign, Location = Grand Mesa:
gm_ts_sites = ['Skyway Open', 'Skyway Tree', 'County Line Open', 'County Line Tree', 'Mesa West Open', 'Mesa West Trees']

In [10]:
# temporal range of GM TS sites: 

dates_dict = {}
for site in gm_ts_sites:
    dates = session.query(LayerData.date).filter(LayerData.site_id == site).distinct().all()
    dates = [d[0] for d in dates] 
    dlist = [str(d) for d in dates]
    # dlist = ", ".join(dlist)
    dates_dict[site] = [dlist[0], dlist[-1]]
    # print('{}: {}\n'.format(site, dlist)) # list all
    
    print('{}\n First: {}\n Last: {}\n\n'.format(site, dlist[0], dlist[-1])) # list first and last

Skyway Open
 First: 2019-12-19
 Last: 2020-04-22


Skyway Tree
 First: 2020-01-22
 Last: 2020-05-03


County Line Open
 First: 2019-12-19
 Last: 2020-04-17


County Line Tree
 First: 2019-12-19
 Last: 2020-03-25


Mesa West Open
 First: 2020-03-05
 Last: 2020-04-09


Mesa West Trees
 First: 2020-03-05
 Last: 2020-04-09




In [ ]:
# Does the lidar overlap with any of these sites?

In [ ]:
# query the sites for temperature
qry = session.query(LayerData).filter(LayerData.site_id.in_(gm_ts_sites)).filter(LayerData.type == 'temperature')
                                      
df = query_to_geopandas(qry, engine)

In [ ]:
# 1. use the list of Time Series sites from above 
sites = gm_ts_sites

# 2. setup the subplot for each site 
fig, axes = plt.subplots(len(sites)%6+1, 6, figsize=(15, 8))

# setup the colorbar
cmap = matplotlib.cm.get_cmap('viridis')

# 3. loop over sites and select the data parameter ('temperature')
for i, site in enumerate(sites):
    
    # query the database by site and for temperature profile data 
    qry = session.query(LayerData).filter(LayerData.site_id.in_([site])).filter(LayerData.type == 'temperature') 
    
    # convert to pandas dataframe  
    df = query_to_pandas(qry, engine)  
    
    # create list of the unique dates (LayerData will have a lot of repeated dates, we only need a list per visit, not per measurement)
    dates = sorted(df['date'].unique())
    
    # grab the plot for this site
    ax = axes.flatten()[i]
    
    # counter to help with plotting
    k=0
    
    
    # 4. loop over dates & plot temperature profile
    for j, date in enumerate(dates):
        
        # grab the temperature profile 
        profile = df[df.date == date]
        
        # don't plot it unless there is data, make sure the dataframe index is > 1
        if len(profile.index) > 0:            
            
            # sort by depth so samples that are taken out of order won't mess up the plot
            profile = profile.sort_values(by='depth')

            # cast as a float; layer profiles are always stored as strings
            profile['value'] = profile['value'].astype(float)

            # plot the temperature profile
            ax.plot(profile['value'], 
                    profile['depth'], 
                    marker='.',
                    color = cmap(k/len(dates)),
                    label=date) 
            # ax.legend()
            
            k+=1 
            
# 5. style the axes
for i, site in enumerate(sites):
    ax = axes.flatten()[i]
    ax.set_xlim(-15, 0)
    ax.set_ylim(-1, 170)
    ax.set_title(sites[i])
    ax.set_xlabel('Temperature [C]')
    ax.set_ylabel('Depth [cm]')

fig.tight_layout()

# close your database session
session.close()     

In [ ]:
# query the sites for temperature
qry = session.query(LayerData).filter(LayerData.site_id.in_(gm_ts_sites)).filter(LayerData.type == 'lwc_vol')
                                      
df = query_to_geopandas(qry, engine)

In [ ]:
# 1. use the list of Time Series sites from above 
sites = gm_ts_sites

# 2. setup the subplot for each site 
fig, axes = plt.subplots(len(sites)%6+1, 6, figsize=(15, 8))

# setup the colorbar
cmap = matplotlib.cm.get_cmap('magma')

# 3. loop over sites and select the data parameter ('temperature')
for i, site in enumerate(sites):
    
    # query the database by site and for temperature profile data 
    qry = session.query(LayerData).filter(LayerData.site_id.in_([site])).filter(LayerData.type == 'lwc_vol') 
    
    # convert to pandas dataframe  
    df = query_to_pandas(qry, engine)  
    
    # create list of the unique dates (LayerData will have a lot of repeated dates, we only need a list per visit, not per measurement)
    dates = sorted(df['date'].unique())
    
    # grab the plot for this site
    ax = axes.flatten()[i]
    
    # counter to help with plotting
    k=0
    
    
    # 4. loop over dates & plot temperature profile
    for j, date in enumerate(dates):
        
        # grab the temperature profile 
        profile = df[df.date == date]
        
        # don't plot it unless there is data, make sure the dataframe index is > 1
        if len(profile.index) > 0:            
            
            # sort by depth so samples that are taken out of order won't mess up the plot
            profile = profile.sort_values(by='depth')

            # cast as a float; layer profiles are always stored as strings
            profile['value'] = profile['value'].astype(float)

            # plot the temperature profile
            ax.plot(profile['value'], 
                    profile['depth'], 
                    marker='.',
                    color = cmap(k/len(dates)),
                    label=date) 
            # ax.legend()
            
            k+=1 
            
# 5. style the axes
for i, site in enumerate(sites):
    ax = axes.flatten()[i]
    ax.set_xlim(-1, 5)
    ax.set_ylim(-10, 170)
    ax.set_title(sites[i])
    ax.set_xlabel('lwc_vol %')
    ax.set_ylabel('Depth [cm]')

fig.tight_layout()

# close your database session
session.close()     

# calculate SWE from depth and density

In [6]:
# sites
sites = gm_ts_sites

new_df = pd.DataFrame()

for i, site in enumerate(sites):
    
    print('COGM site: ', site)
    
    # query the database by site and for density profile data 
    qry = session.query(LayerData).filter(LayerData.site_id.in_([site])).filter(LayerData.type == 'Density') 
    
    # convert to pandas dataframe  
    df = query_to_pandas(qry, engine)
    
    # drop the duplicates (issue to fix in database)
    df = df.drop_duplicates('depth')

    # Convert density to float
    df['value'] = df['value'].astype(float)
    
    # df = df[['depth', 'bottom_depth', 'value']]

    if df['bottom_depth'].iloc[-1] != 0: # if the bottom isn't zero
        df['bottom_depth'].iloc[-1] = 0 # make it zero

    # interpolot where there isn't a density measurement
    df['value'] = df['value'].interpolate()

    # compute SWE!
    df['swe'] = df['value'] * (df['depth'] - df['bottom_depth']) / 100
df



COGM site:  Skyway Open


OperationalError: (psycopg2.OperationalError) SSL SYSCALL error: EOF detected

[SQL: SELECT public.layers.site_name, public.layers.date, public.layers.time_created, public.layers.time_updated, public.layers.id, public.layers.doi, public.layers.date_accessed, public.layers.instrument, public.layers.type, public.layers.units, public.layers.observers, public.layers.latitude, public.layers.longitude, public.layers.northing, public.layers.easting, public.layers.elevation, public.layers.utm_zone, ST_AsEWKB(public.layers.geom) AS geom, public.layers.time, public.layers.depth, public.layers.site_id, public.layers.pit_id, public.layers.bottom_depth, public.layers.comments, public.layers.sample_a, public.layers.sample_b, public.layers.sample_c, public.layers.value, public.layers.flags 
FROM public.layers 
WHERE public.layers.site_id IN (%(site_id_1_1)s) AND public.layers.type = %(type_1)s]
[parameters: {'type_1': 'Density', 'site_id_1_1': 'Skyway Open'}]
(Background on this error at: https://sql

# compute SWE for 1 site, 1 timestamp

In [17]:
# solve swe for one site and time (Skyway 4/22)

dt = datetime.date(2020, 4, 22)

qry = session.query(LayerData)

qry = qry.filter(LayerData.type == 'density').filter(LayerData.date == dt)

qry = qry.filter(LayerData.site_id == 'Skyway Open')

# convert to pandas dataframe  
df = query_to_pandas(qry, engine)

# drop duplicates
df = df.drop_duplicates('depth')

# cast as floats
df['value'] = df['value'].astype(float)

df = df[['depth', 'bottom_depth', 'value', 'site_id']]#, 'pit_id', 'latitude', 'longitude' ]]
# df

if df['bottom_depth'].iloc[-1] != 0: # if the bottom isn't zero
    df['bottom_depth'].iloc[-1] = 0 # make it zero

# interpolot where there isn't a density measurement
df['value'] = df['value'].interpolate() 

# compute SWE!
df['swe'] = df['value'] * (df['depth'] - df['bottom_depth']) / 100
df


# for i, row in df.iterrows:
#     print(row['value'].iloc[i].dtypes)
# for i, row in df['value']:
#     print(type(row['value'][i]))

# if df[-1]['bottom_depth'] !=0:
#     print("bottom isn't 0")

# density condidtions
# for i, row in df.iterrows():
#     print(i, row['bottom_depth'])
#     # if((i == row.shape[0]-1) & (row['bottom_depth'][i] != 0)):
    #     print('yes')
    # print(row['bottom_depth'][i, -1])

# density values ending before 0, make 0 (interpolate with the bottom density)
    # print(row)

# account for missing density values

# 

# swe_lambda = lambda row: row['value'] * (row['depth'] - row['bottom_depth']) / 100
# df['swe'] = df.apply(swe_lambda, axis=1)
# df

,depth,bottom_depth,value,site_id,swe
0,121.0,111.0,383.500000,Skyway Open,38.350000
1,111.0,101.0,357.000000,Skyway Open,35.700000
3,101.0,91.0,391.333333,Skyway Open,39.133333
4,91.0,81.0,403.500000,Skyway Open,40.350000
5,81.0,71.0,347.500000,Skyway Open,34.750000
6,71.0,61.0,372.000000,Skyway Open,37.200000
7,61.0,51.0,311.500000,Skyway Open,31.150000
8,51.0,41.0,326.000000,Skyway Open,32.600000
9,41.0,31.0,325.000000,Skyway Open,32.500000
10,31.0,21.0,349.000000,Skyway Open,34.900000


In [16]:
dt = datetime.date(2020, 4, 22)

qry = session.query(LayerData)

qry = qry.filter(LayerData.type == 'density').filter(LayerData.date == dt)

qry = qry.filter(LayerData.site_id == 'Skyway Open')

# convert to pandas dataframe  
df = query_to_pandas(qry, engine)

# drop duplicates
df = df.drop_duplicates('depth')

# cast as floats
df['value'] = df['value'].astype(float)

df = df[['depth', 'bottom_depth', 'value', 'site_id']]


if df['bottom_depth'].iloc[-1] != 0: # if the bottom isn't zero
    df['bottom_depth'].iloc[-1] = 0 # make it zero

# interpolot where there isn't a density measurement
df['value'] = df['value'].interpolate() 
df

,depth,bottom_depth,value,site_id
0,121.0,111.0,383.500000,Skyway Open
1,111.0,101.0,357.000000,Skyway Open
3,101.0,91.0,391.333333,Skyway Open
4,91.0,81.0,403.500000,Skyway Open
5,81.0,71.0,347.500000,Skyway Open
6,71.0,61.0,372.000000,Skyway Open
7,61.0,51.0,311.500000,Skyway Open
8,51.0,41.0,326.000000,Skyway Open
9,41.0,31.0,325.000000,Skyway Open
10,31.0,21.0,349.000000,Skyway Open


<bound method DataFrame.info of     depth  bottom_depth       value      site_id        swe
0   121.0         111.0  383.500000  Skyway Open  38.350000
1   111.0         101.0  357.000000  Skyway Open  35.700000
3   101.0          91.0  391.333333  Skyway Open  39.133333
4    91.0          81.0  403.500000  Skyway Open  40.350000
5    81.0          71.0  347.500000  Skyway Open  34.750000
6    71.0          61.0  372.000000  Skyway Open  37.200000
7    61.0          51.0  311.500000  Skyway Open  31.150000
8    51.0          41.0  326.000000  Skyway Open  32.600000
9    41.0          31.0  325.000000  Skyway Open  32.500000
10   31.0          21.0  349.000000  Skyway Open  34.900000
11   21.0          11.0  307.000000  Skyway Open  30.700000
12   11.0           0.0  307.000000  Skyway Open  33.770000>

In [ ]:
df

In [ ]:

df['value'] = df['value'].interpolate()
df